In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K

C:\Users\Dragos\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model        

In [3]:

import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [4]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32

print("[INFO] loading images...")
data = []
labels = []

imagePaths = sorted(list(paths.list_images("./images")))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


In [5]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    label = 1 if label == "hotdog" else 0
    labels.append(label)

In [6]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state = 42)
trainY = to_categorical(trainY, num_classes = 2)
testY = to_categorical(testY, num_classes = 2)

In [7]:
aug = ImageDataGenerator(rotation_range = 30, width_shift_range=0.1, 
                         height_shift_range = 0.1, shear_range = 0.2, 
                         zoom_range = 0.2, horizontal_flip=True, fill_mode="nearest")

In [8]:
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy"])

print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
                       validation_data = (testX, testY), steps_per_epoch=len(trainX) // BS,
                        epochs = EPOCHS, verbose = 1
                       )

print("[INFO] serializing network...")
model.save("./hotdog_not_hotdog.model")

[INFO] compiling model...
[INFO] training network...
Epoch 1/25
28/28 [==============================] - 7s 255ms/step - loss: 0.5768 - acc: 0.6644 - val_loss: 0.4095 - val_acc: 0.7987
Epoch 2/25
28/28 [==============================] - 1s 23ms/step - loss: 0.3942 - acc: 0.8342 - val_loss: 0.3111 - val_acc: 0.8864
Epoch 3/25
28/28 [==============================] - 1s 24ms/step - loss: 0.3338 - acc: 0.8611 - val_loss: 0.2945 - val_acc: 0.8766
Epoch 4/25
28/28 [==============================] - 1s 23ms/step - loss: 0.2857 - acc: 0.8758 - val_loss: 0.3688 - val_acc: 0.8247
Epoch 5/25
28/28 [==============================] - 1s 23ms/step - loss: 0.2143 - acc: 0.9192 - val_loss: 0.3647 - val_acc: 0.8377
Epoch 6/25
28/28 [==============================] - 1s 29ms/step - loss: 0.1792 - acc: 0.9271 - val_loss: 0.2736 - val_acc: 0.8799
Epoch 7/25
28/28 [==============================] - 1s 30ms/step - loss: 0.1659 - acc: 0.9431 - val_loss: 0.1735 - val_acc: 0.9221
Epoch 8/25
28/28 [===========

In [9]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Hotdog/Not Hotdog")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("./plots")